In [1]:
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from datetime import datetime
from tqdm import tqdm
import torch.nn as nn
import numpy as np
import os
import torch
import mlflow
import random

In [2]:
# conf.
EXPERIMENT_NAME = "MNIST with CNN v.5"
TRACKING_URL = "http://localhost:5500"
RUN_NAME = f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
RUN_DESCRIPTION = "Trains using the best parameter settings based on 1-epoch validation accuracy (dropout, batch size, learning rate, gamma, step, weight decay)."

In [3]:
def seed_everything(seed: int) -> None:
    """
    Sets random seeds for reproducibility across random, numpy, and PyTorch.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU

In [4]:
class ConvNeuralNet(nn.Module):
    """
    A convolutional neural network for image classification.

    Architecture:
    - Two convolutional blocks with ReLU, MaxPool and Dropout.
    - Fully connected classifier with two hidden layers and dropout.
    - Outputs class logits (e.g. for CrossEntropyLoss).
    """

    def __init__(self, dropout1=0.5, dropout2=0.25):
        super().__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(p=dropout2),
        )

        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=dropout2),
        )

        self.classifier1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 256),
            nn.ReLU(),
            nn.Dropout(p=dropout1),
            nn.Linear(256, 10),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(p=dropout1),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.classifier(x)
        return x

In [5]:
def select_device():
    if torch.cuda.is_available():
        return "cuda"
    elif torch.backends.mps.is_available():
        return "mps"
    else:
        return "cpu"

In [6]:
def save_and_log_model_checkpoint(model, path, artifact_dir):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    torch.save(model.state_dict(), path)
    mlflow.log_artifact(path, artifact_path=artifact_dir)

In [7]:
def run_training_pipeline(params=None):
    """Log model architecture parameters."""

    default_params = {
        "epochs": 100,                 # total training epochs
        "scheduler_step": 1,           # step interval for learning rate scheduler
        "scheduler_gamma": 0.9,        # decay factor for learning rate
        "learning_rate": 0.001,        # initial learning rate
        "weight_decay": 0.00001,       # L2 regularization strength
        "batch_size": 128,             # samples per batch
        "num_workers": 4,              # data loading workers
        "dropout1": 0.5,               # dropout rate for first dropout layer
        "dropout2": 0.25,              # dropout rate for second dropout layer
        "patience": 10,                 # epochs to wait for improvement before early stopping
        "min_delta": 0.0001,           # minimum loss improvement to reset early stopping
        "min_save_accuracy": 99.5,     # minimum accuracy required to save checkpoint
    }

    # update with custom parameters
    if params is not None:
        default_params.update(params)

    # extract parameters
    epochs = default_params["epochs"]
    scheduler_step = default_params["scheduler_step"]
    scheduler_gamma = default_params["scheduler_gamma"]
    learning_rate = default_params["learning_rate"]
    weight_decay = default_params["weight_decay"]
    batch_size = default_params["batch_size"]
    num_workers = default_params["num_workers"]
    dropout1 = default_params["dropout1"]
    dropout2 = default_params["dropout2"]
    patience = default_params["patience"]
    min_delta = default_params["min_delta"]
    min_save_accuracy = default_params["min_save_accuracy"]

    # set up model, optimizer, scheduler and data transforms
    device = select_device()
    model = ConvNeuralNet().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=scheduler_step, gamma=scheduler_gamma)

    transform = transforms.Compose([
        transforms.RandomRotation(12), 
        transforms.RandomResizedCrop(28, scale=(0.9, 1.0)),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.ToTensor(),
    ])

    # load the MNIST dataset and prepare DataLoaders for training and validation
    train_dataset = MNIST(root=".", train=True, transform=transform, download=True)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_dataset = MNIST(root=".", train=False, transform=transforms.ToTensor(), download=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers)

    # set up and start the MLflow experiment run
    mlflow.set_tracking_uri(TRACKING_URL)
    mlflow.set_experiment(EXPERIMENT_NAME)

    with mlflow.start_run(run_name=RUN_NAME):
        run_id = mlflow.active_run().info.run_id
        client = mlflow.tracking.MlflowClient()
        client.set_tag(run_id, "mlflow.note.content", RUN_DESCRIPTION)

        # log the parameters
        all_params = {
            "model_summary": str(model),
            "criterion": "CrossEntropyLoss",
            "optimizer": "ADAM",
            "scheduler_step": scheduler_step,
            "scheduler_gamma": scheduler_gamma,
            "learning_rate": learning_rate,
            "weight_decay": weight_decay,
            "batch_size": batch_size,
            "data_loader_workers": num_workers,
            "dropout1": dropout1,
            "dropout2": dropout2,
            "patience": patience,
            "min_save_accuracy": min_save_accuracy,
        }
        mlflow.log_params(all_params)

        # initialize early stopping counter and best validation loss
        early_stop_counter, best_val_loss = 0, float("inf")

        for epoch in range(1, epochs + 1):
            total_train_loss, total_train_correct, total_train_samples = 0, 0, 0
            total_val_loss, total_val_correct, total_val_samples = 0, 0, 0

            model.train()
            for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}"):
                imgs, labels = imgs.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                total_train_loss += loss.item()
                preds = outputs.argmax(1)
                total_train_correct += (preds == labels).sum().item()
                total_train_samples += labels.size(0)

            avg_train_loss = total_train_loss / len(train_loader)
            train_accuracy = 100 * total_train_correct / total_train_samples

            model.eval()
            with torch.no_grad():
                for imgs, labels in val_loader:
                    imgs, labels = imgs.to(device), labels.to(device)
                    outputs = model(imgs)
                    loss = criterion(outputs, labels)

                    total_val_loss += loss.item()
                    preds = outputs.argmax(1)
                    total_val_correct += (preds == labels).sum().item()
                    total_val_samples += labels.size(0)

            val_loss = total_val_loss / len(val_loader)
            val_accuracy = 100 * total_val_correct / total_val_samples

            print(f"Epoch {epoch}: val_loss={val_loss:.4f}, val_acc={val_accuracy:.2f}")

            # log to MLflow
            mlflow.log_metrics(
                {
                    "loss/train": avg_train_loss,
                    "loss/validate": val_loss,
                    "accuracy/train": train_accuracy,
                    "accuracy/validate": val_accuracy,
                },
                step=epoch,
            )
            scheduler.step()

            
            if (best_val_loss - val_loss) >= min_delta: # significant improvement
                best_val_loss = val_loss
                early_stop_counter = 0
                best_model_path = f"ml_models/{EXPERIMENT_NAME}/{RUN_NAME}/best_model.pt"
                save_and_log_model_checkpoint(model, best_model_path, artifact_dir="best_model")
            else:
                early_stop_counter += 1
            
            if (val_accuracy >= min_save_accuracy and epoch % 3 == 0): # periodic checkpoint
                checkpoint_path = f"ml_models/{EXPERIMENT_NAME}/{RUN_NAME}/epoch_{epoch}.pt"
                save_and_log_model_checkpoint(model, checkpoint_path, artifact_dir="checkpoints")
            
            if early_stop_counter >= patience: # stop if no significant improvement
                last_model_path = f"ml_models/{EXPERIMENT_NAME}/{RUN_NAME}/epoch_{epoch}.pt"
                save_and_log_model_checkpoint(model, last_model_path, artifact_dir="last_model")
                print(f"Early stopping triggered after {epoch} epochs! No improvement for {patience} epochs.")
                break
            
            if epoch == epochs: # save model at final epoch
                last_model_path = f"ml_models/{EXPERIMENT_NAME}/{RUN_NAME}/epoch_{epoch}.pt"
                save_and_log_model_checkpoint(model, last_model_path, artifact_dir="last_model")

In [8]:
# main
print("=" * 50)
print(EXPERIMENT_NAME.upper(), "RUN NAME: ", RUN_NAME.upper())
print("=" * 50)
seed_everything(42)

param_grid = [
    {
        "epochs": 100,
        "dropout1": 0.3,      # Lower dropout rate in first layer
        "dropout2": 0.2,      # Lower dropout rate in second layer
        "batch_size": 128,
        "learning_rate": 0.001,
        "scheduler_gamma": 0.8,
        "scheduler_step": 2,
        "weight_decay": 0.0001,
    },
    {
        "epochs": 100,
        "dropout1": 0.5,      # Original dropout rate
        "dropout2": 0.4,      # Original dropout rate
        "batch_size": 128,
        "learning_rate": 0.0005,  # Lower learning rate
        "scheduler_gamma": 0.8,
        "scheduler_step": 2,
        "weight_decay": 0.0001,
    },
    {
        "epochs": 100,
        "dropout1": 0.6,      # Higher dropout rate
        "dropout2": 0.5,      # Higher dropout rate
        "batch_size": 128,
        "learning_rate": 0.002,   # Higher learning rate
        "scheduler_gamma": 0.8,
        "scheduler_step": 2,
        "weight_decay": 0.0001,
    },
]

for i, params in enumerate(param_grid):
    print(f"\n[RUN] Training config {i+1}/{len(param_grid)}: {params}")
    run_training_pipeline(params)

MNIST WITH CNN V.5 RUN NAME:  RUN_20250424_135605

[RUN] Training config 1/3: {'epochs': 100, 'dropout1': 0.3, 'dropout2': 0.2, 'batch_size': 128, 'learning_rate': 0.001, 'scheduler_gamma': 0.8, 'scheduler_step': 2, 'weight_decay': 0.0001}


2025/04/24 13:56:23 INFO mlflow.tracking.fluent: Experiment with name 'MNIST with CNN v.5' does not exist. Creating a new experiment.
Epoch 1/100: 100%|██████████| 469/469 [00:19<00:00, 24.40it/s]


Epoch 1: val_loss=0.0627, val_acc=97.92


Epoch 2/100: 100%|██████████| 469/469 [00:17<00:00, 26.51it/s]


Epoch 2: val_loss=0.0340, val_acc=98.82


...

Epoch 46: val_loss=0.0112, val_acc=99.64


Epoch 47/100: 100%|██████████| 469/469 [00:17<00:00, 26.14it/s]


Epoch 47: val_loss=0.0110, val_acc=99.64


Epoch 48/100: 100%|██████████| 469/469 [00:17<00:00, 26.09it/s]


Epoch 48: val_loss=0.0109, val_acc=99.63
Early stopping triggered after 48 epochs! No improvement for 10 epochs.
🏃 View run run_20250424_135605 at: http://localhost:5500/#/experiments/386940084818827917/runs/a8418ffe7c494db0b20c6ff8df113ea8
🧪 View experiment at: http://localhost:5500/#/experiments/386940084818827917
